In [1]:
%%bash
date

Thu Jun 22 14:26:20 PDT 2017


In [2]:
%%bash
system_profiler SPSoftwareDataType

Software:

    System Software Overview:

      System Version: Mac OS X 10.7.5 (11G63)
      Kernel Version: Darwin 11.4.2
      Boot Volume: SSD2
      Boot Mode: Normal
      Computer Name: greenbird (2)
      User Name: Sam (Sam)
      Secure Virtual Memory: Enabled
      64-bit Kernel and Extensions: No
      Time since boot: 38 days 3:04



In [3]:
%%bash
#Uses grep to exclude lines that display serial number and hardware UUID
system_profiler SPHardwareDataType | grep -v [SH][ea]



      Model Name: Mac Pro
      Model Identifier: MacPro1,1
      Processor Name: Dual-Core Intel Xeon
      Processor Speed: 3 GHz
      Number of Processors: 2
      Total Number of Cores: 4
      L2 Cache (per Processor): 4 MB
      Memory: 14 GB
      Bus Speed: 1.33 GHz
      Boot ROM Version: MP11.005C.B08
      SMC Version (system): 1.7f10



---

The goal of this notebook is to copy the PacBio fastq.gz files for the oly genome sequencing project to the ```/owl/nightingales/O_lurida``` folder to be in compliance with our data management plan. Since these files do not have unique names (they're all named the same thing, but are stored in different subdirectories), they need to be renamed. Additionally, to confirm that the files were copied and renamed correctly, an md5 checkusm verification needs to take place.

---

### Create md5 checksums of original gzipped fastq files

In [1]:
%%bash
cd /Volumes/owl/nightingales/O_lurida/20170323_pacbio/

#### Use find to locate all the gzipped fastq files, create an md5 checksum, and write the output from each file to a new file

In [2]:
%%bash
time find . -maxdepth 2 -name "*.fastq.gz" -exec md5 {} + > md5checksums_fastq.gz.md5


real	0m0.011s
user	0m0.001s
sys	0m0.005s


Well, that didn't work. I think it didn't change directories. Should have caught this after executing cell #1 above, since there was no output listed. Let's see where I am...

In [3]:
%%bash
pwd

/Users/Sam/GitRepos/LabDocs/jupyter_nbs/sam


Yep, didn't change directories. I think it's because I used the ```bash``` shell magics for the ```cd``` command. Annoying!

In [4]:
cd /Volumes/owl/nightingales/O_lurida/20170323_pacbio/

/Volumes/owl/nightingales/O_lurida/20170323_pacbio


#### Use find to locate all the gzipped fastq files, create an md5 checksum, and write the output from each file to a new file

In [5]:
%%bash
time find . -maxdepth 2 -name "*.fastq.gz" -exec md5 {} + > md5checksums_fastq.gz.md5


real	3m56.425s
user	0m27.842s
sys	0m43.239s


In [6]:
%%bash
cat md5checksums_fastq.gz.md5

MD5 (./170210_PCB-CC_MS_EEE_20kb_P6v2_D01_1/filtered_subreads.fastq.gz) = cc30d6e17499a36960dab140991af9b7
MD5 (./170228_PCB-CC_AL_20kb_P6v2_C01_1/filtered_subreads.fastq.gz) = c0f549d4867c2d45f0a4c5ebb65a7163
MD5 (./170228_PCB-CC_AL_20kb_P6v2_D01_1/filtered_subreads.fastq.gz) = 601c182bceec111f286b1a75f9276613
MD5 (./170228_PCB-CC_AL_20kb_P6v2_E01_1/filtered_subreads.fastq.gz) = 1d8c89d18d976d82f977f53f11657c96
MD5 (./170307_PCB-CC_AL_20kb_P6v2_C01_1/filtered_subreads.fastq.gz) = c000bb09dd10ea85028cd73ba1f8ce51
MD5 (./170307_PCB-CC_AL_20kb_P6v2_C02_1/filtered_subreads.fastq.gz) = 7f387f334adad2c11f71c4216b0d8fe2
MD5 (./170314_PCB-CC_20kb_P6v2_A01_1/filtered_subreads.fastq.gz) = c21ce30b76f57485a3cbe8712a623bd5
MD5 (./170314_PCB-CC_20kb_P6v2_A02_1/filtered_subreads.fastq.gz) = 20b0b0743fe0214bde319d2d40a40067
MD5 (./170314_PCB-CC_20kb_P6v2_A03_1/filtered_subreads.fastq.gz) = 511866d0fc60fe3e8bb6dea4ce87a64f
MD5 (./170314_PCB-CC_20kb_P6v2_A04_1/filtered_subreads.fastq.gz) = 206b30a4b23

### Copy, rename, and generate new md5 checksums

In [7]:
%%bash
# Find all xml files and store results in array
xml_array=($(find . -maxdepth 2 -name "*.xml"))

# Print contents of array, with some formatting for easier reading.
echo "Contents of xml_array:"
printf '%s\n' "${xml_array[@]}"
echo ""
echo "-------------"

# Find all fastq.gz files and store results in array
fastq_array=($(find . -maxdepth 2 -name "*.fastq.gz"))
echo "Contents of xml_array:"
printf '%s\n' "${fastq_array[@]}"
echo ""
echo "-------------"

# Use parameter expansion to remove path from each component in fastq_array.
# Store results in new array.
fastq_nopath_array=($(echo "${fastq_array[@]##*/}"))

# Print contents of array, with some formatting for easier reading.
echo "Contents of fastq_nopath_array:"
printf '%s\n' "${fastq_nopath_array[@]}"
echo ""
echo "-------------"

# Use parameter expansion to remove path from each component in xml_array.
# Store results in new array.
xml_nopath_array=($(echo "${xml_array[@]##*/}"))

# Print contents of array, with some formatting for easier reading.
echo "Contents of xml_nopath_array:"
printf '%s\n' "${xml_nopath_array[@]}"
echo ""
echo "-------------"

# Use parameter expansion to remove the suffix (.xml) from each component in xml_nopath_array.
# Store results in new array.
xml_nosuffix_array=($(echo "${xml_nopath_array[@]%%.*}"))

# Print contents of array, with some formatting for easier reading.
echo "Contents of xml_nosuffix_array:"
printf '%s\n' "${xml_nosuffix_array[@]}"
echo ""
echo "-------------"

# Loop through each index (i.e. the number corresponding to each element in the array).
# Using the paths to each fastq.gz stored in the fastq_array, copy the fastq.gz file to the O_lurida nigthtingales folder &
# use array elements to provide new name to copied file.
# List the newly copied/named file to verify it got copied/renamed.
# Create md5 checksums for each newly copied/renamed file and append to checksums.md5 file.
# Use grep to verify info was written to checksums.md5 file
for item in "${!fastq_array[@]}"; do
    cp "${fastq_array[$item]}" /Volumes/owl/nightingales/O_lurida/"${xml_nosuffix_array[$item]}_${fastq_nopath_array[$item]}"
    ls /Volumes/owl/nightingales/O_lurida/"${xml_nosuffix_array[$item]}_${fastq_nopath_array[$item]}"
    md5 /Volumes/owl/nightingales/O_lurida/"${xml_nosuffix_array[$item]}_${fastq_nopath_array[$item]}" >> \
    /Volumes/owl/nightingales/O_lurida/checksums.md5
    grep "${xml_nosuffix_array[$item]}_${fastq_nopath_array[$item]}" /Volumes/owl/nightingales/O_lurida/checksums.md5
done

Contents of xml_array:
./170210_PCB-CC_MS_EEE_20kb_P6v2_D01_1/m170211_224036_42134_c101073082550000001823236402101737_s1_X0.metadata.xml
./170228_PCB-CC_AL_20kb_P6v2_C01_1/m170301_100013_42134_c101174162550000001823269408211761_s1_p0.metadata.xml
./170228_PCB-CC_AL_20kb_P6v2_D01_1/m170301_162825_42134_c101174162550000001823269408211762_s1_p0.metadata.xml
./170228_PCB-CC_AL_20kb_P6v2_E01_1/m170301_225711_42134_c101174162550000001823269408211763_s1_p0.metadata.xml
./170307_PCB-CC_AL_20kb_P6v2_C01_1/m170308_163922_42134_c101174252550000001823269408211742_s1_p0.metadata.xml
./170307_PCB-CC_AL_20kb_P6v2_C02_1/m170308_230815_42134_c101174252550000001823269408211743_s1_p0.metadata.xml
./170314_PCB-CC_20kb_P6v2_A01_1/m170315_001112_42134_c101169372550000001823273008151717_s1_p0.metadata.xml
./170314_PCB-CC_20kb_P6v2_A02_1/m170315_063041_42134_c101169382550000001823273008151700_s1_p0.metadata.xml
./170314_PCB-CC_20kb_P6v2_A03_1/m170315_124938_42134_c101169382550000001823273008151701_s1_p0.metad

Wow! I'm impressed with myself! It nearly all came out as intended!! Forgot to change the second ```echo``` statement to read "Contents of fastq_array:". Other than that, it all worked. Feels good!

### Compare md5 checksums

#### Need to compare original checksums with that of the copied/renamed files to ensure file integrity didn't change during the copying process.

Quick test of using ```grep``` and ```awk``` to isolate just the checksum values...

In [8]:
grep "filtered_subreads.fastq.gz" /Volumes/owl/nightingales/O_lurida/checksums.md5 | awk '{print "$4"}'

SyntaxError: invalid syntax (<ipython-input-8-0e82f528bf91>, line 1)

In [9]:
%%bash
grep "filtered_subreads.fastq.gz" /Volumes/owl/nightingales/O_lurida/checksums.md5 | awk '{print "$4"}'

$4
$4
$4
$4
$4
$4
$4
$4
$4
$4


In [10]:
%%bash
grep "filtered_subreads.fastq.gz" /Volumes/owl/nightingales/O_lurida/checksums.md5 | awk '{print $4}'

cc30d6e17499a36960dab140991af9b7
c0f549d4867c2d45f0a4c5ebb65a7163
601c182bceec111f286b1a75f9276613
1d8c89d18d976d82f977f53f11657c96
c000bb09dd10ea85028cd73ba1f8ce51
7f387f334adad2c11f71c4216b0d8fe2
c21ce30b76f57485a3cbe8712a623bd5
20b0b0743fe0214bde319d2d40a40067
511866d0fc60fe3e8bb6dea4ce87a64f
206b30a4b231541a85d4707f0b907153


Well, I just looked through some previously used code and realized I didn't need to perform the above grep/awk test.

In [11]:
%%bash
original_md5=($(awk '/filtered_subreads.fastq.gz/{print $4}' /Volumes/owl/nightingales/O_lurida/20170323_pacbio/md5checksums_fastq.gz.md5))
current_md5=($(awk '/filtered_subreads.fastq.gz/{print $4}' /Volumes/owl/nightingales/O_lurida/checksums.md5
for ((i=0;i<=$count;++i))
    do
        printf "%s\n" "${original_md5[$i]}"
        printf "%s\n\n" "${current_md5[$i]}"
    done

bash: line 2: unexpected EOF while looking for matching `)'
bash: line 8: syntax error: unexpected end of file


In [13]:
%%bash
original_md5=($(awk '/filtered_subreads.fastq.gz/{print $4}' /Volumes/owl/nightingales/O_lurida/20170323_pacbio/md5checksums_fastq.gz.md5))
current_md5=($(awk '/filtered_subreads.fastq.gz/{print $4}' /Volumes/owl/nightingales/O_lurida/checksums.md5))
count=$(( ${#original_md5[@]} - 1 ))
for ((i=0;i<=$count;++i))
    do
        printf "%s\n" "${original_md5[$i]}"
        printf "%s\n\n" "${current_md5[$i]}"
    done

cc30d6e17499a36960dab140991af9b7
cc30d6e17499a36960dab140991af9b7

c0f549d4867c2d45f0a4c5ebb65a7163
c0f549d4867c2d45f0a4c5ebb65a7163

601c182bceec111f286b1a75f9276613
601c182bceec111f286b1a75f9276613

1d8c89d18d976d82f977f53f11657c96
1d8c89d18d976d82f977f53f11657c96

c000bb09dd10ea85028cd73ba1f8ce51
c000bb09dd10ea85028cd73ba1f8ce51

7f387f334adad2c11f71c4216b0d8fe2
7f387f334adad2c11f71c4216b0d8fe2

c21ce30b76f57485a3cbe8712a623bd5
c21ce30b76f57485a3cbe8712a623bd5

20b0b0743fe0214bde319d2d40a40067
20b0b0743fe0214bde319d2d40a40067

511866d0fc60fe3e8bb6dea4ce87a64f
511866d0fc60fe3e8bb6dea4ce87a64f

206b30a4b231541a85d4707f0b907153
206b30a4b231541a85d4707f0b907153



Great! Visual inspection indicates the MD5 checkums have not changed during the copying/renaming process!

#### Explanation of the above command:

##### The gist is that the output from the awk command is saved to an array. Then a for loop is run which iterates over the array and prints the element at each position.

---
###### Break down the first line:

```original_md5=()``` - This is an empty array called "original_md5".

```$()``` - This is an empty command substitution. The stdout of commands within the parentheses are stored.

```awk '/filtered_subreads.fastq.gz/{print $4}' md5_file``` - Awk looks for any lines from the input file (md5_file) with "filtered_subreads.fastq.gz" in them. If a line contains "filtered_subreads.fastq.gz", awk prints the fourth field (i.e. the fourth column).

Summary - The output from each result printed by awk is saved in an auto-incrementing fashion in the array called "original_md5".

---
###### Break down the 3rd line:

```count=$(())``` - A variable called "count". This is a combination of empty command substitution and bash arithmeetic. Double parentheses are required for bash arithmetic.


```${#current_md5[@]} - 1``` - This prints the number of indices (#) in the array called "original_md5" and subtracts 1 from that number. Subtraction of one is necessary because bash is a zero-based language (e.g. the array starts at index 0).

Summary - The length of the array minus one is saved to the variable called "count".

---
###### Break down the for loop:

```((i=0;i<=$count;++i))``` - Sets variable "i" to 0. Then, the loop evaluates whether or not the value of "i" is less than/equal to the value in the variable "count". If that condition is met, the loop increases the value stored in "i" by 1 and continues through the loop.

```printf "%s\n" "${original_md5[$i]}"``` - Prints the value at the array index designated by the value currently stored in "i" (the printing is specified by the "%s", which means string). This is followed by printing a new line (\n).

Summary - This prints the value at each position within the array and uses printf to improve legibility of output.